<a href="https://colab.research.google.com/github/rahulgundala007/NLP_text_summarization/blob/main/GPT_2_saved_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 18.5 MB/s eta 0:00:00


In [3]:
import fitz  # PyMuPDF

def clean_text(text):
    # Replace any sequence of whitespace characters with a single space
    return ' '.join(text.split())

# Function to extract and clean text from PDF
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    with fitz.open(pdf_path) as pdf:
        text = ""
        # Iterate over each page
        for page_num in range(len(pdf)):
            # Extract text from each page
            page_text = pdf[page_num].get_text()
            # Add cleaned page text to the combined text
            text += clean_text(page_text)
    return text

# Specify the path to your PDF file
pdf_path = '/content/drive/MyDrive/SavedModel/ToS_document/samsung.pdf'  # Replace with your actual path to the PDF
# Extract and clean text from the PDF
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

Samsung Business Services Terms and Conditions (USA) The sale and performance of Samsung Business Services is governed by these Samsung Business Services Terms and Conditions (the "Terms and Conditions"). The Terms and Conditions apply to all: 1) enterprise purchases of Services (as defined below) directly from Samsung; and 2) enterprise purchases of Services through an authorized Samsung reseller or distributor (“reseller”). By placing your Order (as defined below) for the Services, receiving delivery of the Services, utilizing the Services, or clicking/checking any “I Agree” button or box or similar on the Samsung Business Services website in connection with your purchase, you agree to be bound by these Terms and Conditions. These Terms and Conditions are agreed between (i) you, either individually if you are agreeing to it in your own capacity, or if you are authorized to agree to these terms on behalf of your company or another entity, between the entity for whose benefit you act (

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model_path = "/content/drive/MyDrive/SavedModel/GPT-2"
tokenizer_path = "/content/drive/MyDrive/SavedModel/GPT-2"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)


In [14]:
def prepare_text(text, tokenizer):
    # Prepend the prompt to the text
    prompt = "summarize: " + text
    # Encode the text using the tokenizer
    encoded_input = tokenizer(prompt, return_tensors='pt', max_length=1024, truncation=True, padding="max_length")
    return encoded_input

def generate_summary(text, model, tokenizer):
    # Prepare the text
    encoded_input = prepare_text(text, tokenizer)

    # Generate output using the model
    output_sequences = model.generate(
        input_ids=encoded_input['input_ids'],  # input tokens
        attention_mask=encoded_input['attention_mask'],  # attention masks
        max_length=5000,  # maximum length of the output tokens
        min_length=500,
        num_return_sequences=1,  # number of output sequences to generate
        no_repeat_ngram_size=2,  # avoid repetition
        early_stopping=True  # stop once model predicts the eos_token
    )

    # Decode the output sequences to text
    summary = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return summary


In [15]:
# Example text to summarize
text = extracted_text

# Generate the summary
summary = generate_summary(text, model, tokenizer)
print("Summary:", summary)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Summary: summarize: Samsung Business Services Terms and Conditions (USA) The sale and performance of Samsung Business Services is governed by these Samsung Business Services Terms and Conditions (the "Terms and Conditions"). The Terms and Conditions apply to all: 1) enterprise purchases of Services (as defined below) directly from Samsung; and 2) enterprise purchases of Services through an authorized Samsung reseller or distributor (“reseller”). By placing your Order (as defined below) for the Services, receiving delivery of the Services, utilizing the Services, or clicking/checking any “I Agree” button or box or similar on the Samsung Business Services website in connection with your purchase, you agree to be bound by these Terms and Conditions. These Terms and Conditions are agreed between (i) you, either individually if you are agreeing to it in your own capacity, or if you are authorized to agree to these terms on behalf of your company or another entity, between the entity for who

In [16]:
pip install PyPDF2 transformers sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.9 MB/s eta 0:00:00


In [17]:
import sacrebleu

def compute_bleu(generated_summary, reference_summary):
    bleu_score = sacrebleu.corpus_bleu([generated_summary], [[reference_summary]])
    return bleu_score.score

In [22]:
import sacrebleu

def compute_bleu(output_summary, reference_summary):
    # BLEU score expects a list of references and a hypothesis
    bleu_score = sacrebleu.corpus_bleu([output_summary], [[reference_summary]])
    return bleu_score.score

# Example reference summary
reference_summary = "Certainly! Here's the text without quotes: The following are the terms and conditions that govern the sale and performance of Samsung Business Services, which include the provision of services directly from Samsung and through authorized resellers. The term Services refers to each component of the Samsung enterprise service offering, as described in a service guide (each, a service order), with specific terms for each component. These terms and conditions apply to all enterprise purchases of services from Samsung and reseller purchases of services through Samsung's authorized reseller, with final pricing and sales terms between the reseller and the purchaser determined by the seller. The Services and Deliverables are intended for internal business use unless permitted by Samsung in writing. Here are the key points to keep in mind: 1. Service Description: This section describes the services and deliverables offered by Samsung. 2. Payment Terms: These terms are determined by Samsung, with specific payment terms for services purchased directly from the company. 3. Availability: The terms used in this section are specific to the service guide or Order, with the meanings defined by the specific Order. 4. Service Availability: Each Samsung Service Guide or Order describes: (i) Available service, and (ii) Deliverables. 5. Terms Not Specified: Capitalized terms used but not defined in the terms and Conditions shall have the meanings as set forth in the applicable Order. 6. Service Terms: Here's a summary of the key terms used within the terms, including service descriptions, payment terms, and terms for service purchases, including reseller terms and terms applicable to Samsung's service offerings. 7. Business Services: The sale and performance of Samsung Services is governed by these terms, which are agreed between you and Samsung in your own capacity, between the entity for whose benefit you act, and Samsung Electronics America, Inc. (Samsung), which is responsible for service provision and service management. 8. Business Use: The Services are meant for internal use, with limited external use for external use. 9. Payment Terms: The payment terms that apply to you if Services are purchased through a reseller differ from those for directly purchased from Samsung. 10. Services Availability: Includes the terms for individual purchases, with detailed descriptions of the services provided by Samsung within the Service Guide and Order. 11. Services: Includes a detailed description of each component, including pricing, delivery, and use terms, applicable to all users, within the scope of the service agreement."

# Example generated summary (replace this with your actual generated summary)
generated_summary = summary

# Calculate the BLEU score
bleu_score = compute_bleu(generated_summary, reference_summary)
print(f"BLEU Score: {bleu_score}")


BLEU Score: 14.71468207052418


In [25]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d649e13d943b0834a912c4fcf6e183abba4c160be21b40d53974ead4f136cdb3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [29]:
from rouge_score import rouge_scorer

def compute_rouge(output_summary, reference_summaries):
    # Create a ROUGE scorer, can specify which ROUGE metrics to calculate
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, output_summary) for ref in reference_summaries]

    # Aggregate scores (average)
    aggregated_scores = {}
    for key in scores[0].keys():
        aggregated_scores[key] = {
            'precision': sum([score[key].precision for score in scores]) / len(scores),
            'recall': sum([score[key].recall for score in scores]) / len(scores),
            'fmeasure': sum([score[key].fmeasure for score in scores]) / len(scores)
        }
    return aggregated_scores

# Example usage
reference_summaries = [
    "Certainly! Here's the text without quotes: The following are the terms and conditions that govern the sale and performance of Samsung Business Services, which include the provision of services directly from Samsung and through authorized resellers. The term Services refers to each component of the Samsung enterprise service offering, as described in a service guide (each, a service order), with specific terms for each component. These terms and conditions apply to all enterprise purchases of services from Samsung and reseller purchases of services through Samsung's authorized reseller, with final pricing and sales terms between the reseller and the purchaser determined by the seller. The Services and Deliverables are intended for internal business use unless permitted by Samsung in writing. Here are the key points to keep in mind: 1. Service Description: This section describes the services and deliverables offered by Samsung. 2. Payment Terms: These terms are determined by Samsung, with specific payment terms for services purchased directly from the company. 3. Availability: The terms used in this section are specific to the service guide or Order, with the meanings defined by the specific Order. 4. Service Availability: Each Samsung Service Guide or Order describes: (i) Available service, and (ii) Deliverables. 5. Terms Not Specified: Capitalized terms used but not defined in the terms and Conditions shall have the meanings as set forth in the applicable Order. 6. Service Terms: Here's a summary of the key terms used within the terms, including service descriptions, payment terms, and terms for service purchases, including reseller terms and terms applicable to Samsung's service offerings. 7. Business Services: The sale and performance of Samsung Services is governed by these terms, which are agreed between you and Samsung in your own capacity, between the entity for whose benefit you act, and Samsung Electronics America, Inc. (Samsung), which is responsible for service provision and service management. 8. Business Use: The Services are meant for internal use, with limited external use for external use. 9. Payment Terms: The payment terms that apply to you if Services are purchased through a reseller differ from those for directly purchased from Samsung. 10. Services Availability: Includes the terms for individual purchases, with detailed descriptions of the services provided by Samsung within the Service Guide and Order. 11. Services: Includes a detailed description of each component, including pricing, delivery, and use terms, applicable to all users, within the scope of the service agreement.",
    "Samsung Business Services Terms and Conditions (the Terms and Conditions) govern the sale and performance of Samsung Business Services, including enterprise purchases and reseller purchases. As used herein, “ Services” means each component of the Samsung enterprise service offering as described in a service guide (each, a service order), with the terms and conditions set forth in the applicable Order. Specific terms used within these terms are governed by specific conditions within Samsung's agreement with the user, which is governed by a specific agreement between the user and Samsung. The terms and Conditions apply to all: 1) enterprise purchases �� of S ervices  (as defined below) directly from Samsung; and 2) enterprise SourceFilepurchases  of Services t hrough a Samsung reseller, with final pricing and sales terms between the reseller and the customer. The Services and Deliverables are intended for internal business use unless permitted by Samsung in writing, under which case they are sold through Samsung's authorized reseller or distributor, with payment terms to be determined by the seller directly from the service provider. Here are the key points: 1. Service Description. 2. Payment Terms. 3. Availability of Services. 4. Service Availability. 5. Availability. References to specific terms in the Order. Note: Terms not defined in the Terms may differ from the Order, but should have the meanings as set forth within the Order within the scope of this section. 1. Services Description: Includes a detailed description of the services offered by Samsung, including pricing, delivery terms, and use conditions for the service, including warranties, warranties, and terms for Samsung's reseller agreement.2. Payments Terms: Includes the payment terms that apply to you if Services are purchased directly fromÂ Samsung.3. Availability: Includes specific terms within the Service Guide or Order, with specific conditions for use of the service.Â References to terms not defined within the Terms.Â Includes:Â Services Overview.Â Payment terms.Â Providing Services.Â Services Availability.Â License Agreement.Â Compliance.Â Governance.Â Entrepreneurship.Â Banking.Â Taxes.Â Final Sale Proceeding.Â Overview. Samsung Services Offerings.Â Sale Overview. Key Points:Â Business Services Governance;Â ServicesÂ Overview;Â Payment Terms;Â References.Â Settlement Process.Â Violated Terms:Â Settlement Terms. **Automated Payment Practices**.Â WARRANT WARRANT WARRANTIES.Â SERVICES Overview.",
    "Samsung Business Services Terms and Conditions (USA) The sale and performance of Samsung Business Services is governed by these Samsung Business Services Terms and Conditions (the Terms and Conditions). The Terms and Conditions apply to all: 1) enterprise purchases of Services (as defined below) directly from Samsung; and 2) enterprise purchases of Services through an authorized Samsung reseller or distributor (“reseller”). By placing your Order (as defined below) for the Services, receiving delivery of the Services, utilizing the Services, or clicking/checking any “I Agree” button or box or similar on the Samsung Business Services website in connection with your purchase, you agree to be bound by these Terms and Conditions. These Terms and Conditions are agreed between (i) you, either individually if you are agreeing to it in your own capacity, or if you are authorized to agree to these terms on behalf of your company or another entity, between the entity for whose benefit you act (in either case, you, your or Customer), and (ii) Samsung Electronics America, Inc. (“Samsung”). The Services and Deliverables are solely for your internal business use unless permitted by Samsung in writing. If you purchased Services through a reseller, final prices and sales terms will be between you and the reseller; however, these Terms and Conditions apply to Samsung's provision of the Services to you. Section 3 below sets forth the payment terms that apply to you if Services are purchased directly from Samsung. Capitalized terms used but not defined in these Terms and Conditions shall have the meanings as set forth in the applicable Order. 1. Service Descriptions. As used herein, “Services” means each Samsung enterprise service offering as described in a service guide (each, a “Samsung Service Guide”) or an Order. Each Samsung Service Guide or Order describes: (i) available service options; (ii) specific projects, tasks, functions, responsibilities and services to be furnished by Samsung to Customer; (iii) deliverables to be provided (the “Deliverables”); (iv) assumptions, conditions and dependencies impacting the Services or upon which the pricing or delivery of the Services might be based; (v) the charges and expenses for the Services (“Fees”); (vi) additional terms and conditions that apply to the particular Services (for example, service levels, performance requirements, milestones, project plans, acceptance testing, and license rights for Deliverables); and (vii) the duration of the Services (the “Service Term”). 2. Services. On the Service Start Date (as defined below), Samsung shall commence providing Customer with the Services set forth in the Order, and Customer shall commence with the performance of its obligations thereunder. Unless otherwise set forth in an Order, Customer’s acceptance of the Services occurs upon performance of the Services by Samsung. A. Services Manager. All communications regarding any Order shall be directed to the applicable Customer or Samsung contacts. The Samsung and Customer contacts shall have the authority to act on behalf of the respective party in all day-to-day matters pertaining to the applicable Order, but shall not have the authority, through action, inaction or otherwise, to amend or modify these Terms and Conditions or an Order. Either party may change its designated contact upon written notice to the other party. B. Service Locations. Samsung may provide the Services (or portions thereof) from any location determined by Samsung or relocate the provision of the Services (or any portion thereof). C. Access & Use. Customer acknowledges that it is responsible (administratively and financially) for obtaining all required approvals, licenses, authorizations, consents, and permits for Samsung to use or access the Customer sites or Customer contractor sites, with such sites being kept free of health and safety hazards. D. Changes to Scope of Services. If the applicable Samsung Service Guide or Order permits change requests, either Samsung or Customer may request changes to the then existing scope of Services or Deliverables, including new or additional services, in accordance with the following change control procedures. All change requests will: (i) be in writing; (ii) have a description of the requested change; (iii) describe the purpose for the change; (iv) determine the priority of the change; (v) reflect the date of requested implementation; (vi) identify the pricing impact of the change, if any; and (vii) give the name of the individual authorized to request such changes. Changes requested will be reviewed by the receiving party as soon as possible, and all changes mutually agreed to by the parties will become effective upon the date set forth in the document executed by both parties describing such changes. E. Customer Obligations. Customer acknowledges that the ability of Samsung to provide certain Services and Deliverables in accordance with these Terms and Conditions and the associated Service Guide or Order (including the agreed pricing and delivery models) are contingent upon, and Customer represents, warrants and covenants that",
]
generated_summary = summary

# Compute ROUGE scores
rouge_scores = compute_rouge(generated_summary, reference_summaries)
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': {'precision': 0.5695142378559465, 'recall': 0.8257187932644322, 'fmeasure': 0.6524538001043724}, 'rouge2': {'precision': 0.46624737945492667, 'recall': 0.6112613862301752, 'fmeasure': 0.5131723076724933}, 'rougeL': {'precision': 0.44932998324958123, 'recall': 0.5765639589169002, 'fmeasure': 0.4904461795779201}}
